# NBS 产品爬虫--Trial Codes
``By Zhiyuan Chen``

In [1]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import csv
import pandas as pd

In [2]:
# define the function of retrying requesting the website
import random
import urllib.request
headers =  ('User-Agent','Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11')
opener = urllib.request.build_opener()
opener.addheaders  = [headers]

def reopen(url, timeout, maxt):
    """
    This function reopens the url under random sleeping time, the sleep time increases
    as more trials are called 
    ---------------------------------------------------------------------------------
    Var Def:
    url: objective url
    timeout: maximum time allowed for opening the url
    longsleep: the time for a long sleep after failed trials
    maxt: maximum number of trials
    maxpause: maximum number of pauses (long sleep)
    """
    fails = 0
    pause = 0
    while True:
        try:
            if fails >= maxt:
                #if pause >= maxpause:
                #    print('Dead Url:'+url)
                #    print('Number of Pauses:', maxpause)
                #    break
                #print('Getting a pause of ', longsleep,'s...')
                #time.sleep(longsleep + random.random())
                #pause +=1
                #html = opener.open(url, None, timeout)
                #return html
                print('Dead Url:'+url)
                break
            html = opener.open(url, None, timeout)
            return html
        except:
            fails +=1
            stime = fails + random.random() # sleeping time increases as we try more
            time.sleep(stime)
            print ('connection timeout, retrying:', fails)
            print ('sleeping time:', stime)
        else: 
            print(url)
            break 

In [3]:
# define the function of retrieving hrefs and texts
#-------------------------------------------------
def reftextRetrieve(item):
    if item.select('a')!=[]:
        item_a = item.select('a')
        href = item_a[0].get('href')
        return item_a, href
    else:
        if item.select('td')!=[]:
            item_a = item.select('td')
            href = []
            return item_a, href
        else:
            print('EEOR!: selected invalid tag') 

In [4]:
# extract the first page with two-digit products

my_url = 'http://www.stats.gov.cn/tjsj/tjbz/tjypflml/index.html'
html = uReq(my_url)
page_soup = soup(html.read(), 'html.parser')

# find each name of the product catalog
containers = page_soup.findAll("span", {"class":"cont_tit"})
code_product = []
for item in containers:
    item_list = item.findAll('font', {'class':'cont_tit03'})
    name = item_list[0].text
    code_product.append([name[0:2], name[3:]])

In [8]:
# reopen if timeout
# random sleeping time after each trial
# a long pause after a sequence of trials
import urllib.request
import random
import time
t0 = time.time()
# settings for the reopen() function
#------------------------------------------------------
headers =  ('User-Agent','Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11')
opener = urllib.request.build_opener()
opener.addheaders  = [headers]
maxtr = 10
timeout= 10

#------------------------------------------------------
# index page 1
tags_index = [code_product[i][0] for i in range(len(code_product))] # tags for index html links
url_index = []
html_main = 'http://www.stats.gov.cn/tjsj/tjbz/tjypflml/2010/'
for tag in tags_index:
    link = html_main + tag +'.html'
    url_index.append(link)

#Page 1#
for i, url in enumerate([url_index[0]]):
    code2, pdt2 = code_product[i]  # two-digit products
    html = reopen(url, maxtr, timeout)
    page_sp = soup(html.read(), 'html.parser')
    pdt_containers = page_sp.find_all("tr", {"class": "citytr"})

id = 18
item = pdt_containers[id]
item_a, href1 = reftextRetrieve(item)

if item_a!=[]:
   code4, pdt4 = [item_a[0].text, item_a[1].text]
   if href1 ==[]:
      code6, pdt6 = ['', '']
      code8, pdt8 = ['', '']
      code10,pdd10= ['', ''] 
      
   else:
       url1 = html_main + href1 #hyperlinks to subcategories of 6 digit
       html1 = reopen(url1, maxtr, timeout)
       page_sp1 = soup(html1.read(), 'html.parser')
       pdt_containers1 = page_sp1.find_all("tr", {"class": "countytr"})
       for item in pdt_containers1:
            item_a1, href2 = reftextRetrieve(item)
            if item_a1!=[]:
               code6, pdt6 = [item_a1[0].text, item_a1[1].text]
               if href2==[]:
                  code8, pdt8 = ['', '']
                  code10,pdd10= ['', '']
                                 
               else: 
                   url2 = html_main + code2 +'/' + href2 # hyperlinks to 8-digit
                   html2 = reopen(url2, maxtr, timeout)
                   page_sp2 = soup(html2.read(), 'html.parser')
                   pdt_containers2 = page_sp2.find_all("tr", {"class": "towntr"})
                   if pdt_containers2!=[]:
                       for item in pdt_containers2:
                           item_a2, href3 = reftextRetrieve(item)
                           if item_a2!=[]:
                               code8, pdt8 = [item_a2[0].text, item_a2[1].text]
                               if href2 == []:
                                 code10, pdt10 = ['', '']
                                 
                               else:
                                  url3 = html_main + '/'.join([code4[0:2], code4[2:4]]) + '/'+ href3
                                  html3 = reopen(url3, maxtr, timeout)
                                  page_sp3 = soup(html3.read(), 'html.parser')
                                  pdt_containers3 = page_sp3.find_all("tr", {"class": "villagetr"})
                                  for item in pdt_containers3:
                                        item_a3 = item.select('td')
                                        code10, pdt10 = [item_a3[0].text, item_a3[1].text]
                                        print('code10:' + code10)
                                        
                   else:
                        code8, pdt8 = ['','']
                        pdt_containers3 = page_sp2.find_all("tr", {"class": "villagetr"})
                        for item in pdt_containers3:
                            item_a3 = item.select('td')
                            code10, pdt10 = [item_a3[0].text, item_a3[1].text]
                            print('code10:' + code10)
                            


code10:0119010000
code10:0119020000
code10:0119030000
code10:0119040000
code10:0119050000
code10:0119060000
code10:0119070000
code10:0119080000
code10:0119090000
code10:0119100000
code10:0119110000
code10:0119120000
code10:0119130000
code10:0119140000
code10:0119150000
code10:0119160000
code10:0119170000
code10:0119180000
code10:0119190000
code10:0119200000
code10:0119210000
code10:0119220000
code10:0119230000
code10:0119240000
code10:0119250000
code10:0119260000
code10:0119270000
code10:0119280000
code10:0119290000
code10:0119300000
code10:0119310000
code10:0119320000
code10:0119330000
code10:0119340000
code10:0119350000
code10:0119360000
code10:0119370000
code10:0119380000
code10:0119390000
code10:0119400000
code10:0119410000
code10:0119420000
code10:0119430000
code10:0119440000
code10:0119450000
code10:0119460000
code10:0119470000
code10:0119480000
code10:0119490000
code10:0119990000
